In [4]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TFAutoModel, AutoTokenizer

In [5]:
df = pd.read_csv('/content/olid-training-v1.0.tsv', sep='\t')

# Remove unnecessary columns
df = df.drop(['id', 'subtask_a', 'subtask_c'], axis=1)

# Drop missing values
df = df.dropna()

# Encode the labels
labels = {'TIN': 0, 'UNT': 1}
df['subtask_b'] = df['subtask_b'].apply(lambda x: labels[x])

In [6]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['tweet'], df['subtask_b'], test_size=0.2, random_state=42)

In [7]:
# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [8]:
# Tokenize the training and validation sets
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

In [9]:

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val.values
))

In [10]:
# Batch the datasets
train_dataset = train_dataset.batch(16)
val_dataset = val_dataset.batch(16)

In [11]:
# Define the base model
base_model = TFAutoModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [12]:
# Define the meta-learner model
meta_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [13]:
# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [14]:
# Define the training loop
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        base_output = base_model(inputs)['last_hidden_state'][:, 0, :]
        meta_output = meta_model(base_output)
        loss = loss_fn(labels, meta_output)
    gradients = tape.gradient(loss, meta_model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, meta_model.trainable_weights))
    return loss

In [15]:
# Train the model
for epoch in range(10):
    for inputs, labels in train_dataset:
        loss = train_step(inputs, labels)
    print('Epoch', epoch+1, '- Loss:', loss.numpy())

Epoch 1 - Loss: 0.2633015
Epoch 2 - Loss: 0.23909459
Epoch 3 - Loss: 0.23403917
Epoch 4 - Loss: 0.2312129
Epoch 5 - Loss: 0.22886568
Epoch 6 - Loss: 0.22676066
Epoch 7 - Loss: 0.22475609
Epoch 8 - Loss: 0.22277167
Epoch 9 - Loss: 0.2210093
Epoch 10 - Loss: 0.21948808


In [16]:
# Evaluate the model on the validation set
true_labels = []
predictions = []
for inputs, labels in val_dataset:
    base_output = base_model(inputs)['last_hidden_state'][:, 0, :]
    meta_output = meta_model(base_output).numpy()
    predicted_labels = np.argmax(meta_output, axis=1)
    true_labels.extend(labels.numpy())
    predictions.extend(predicted_labels)

from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93       768
           1       0.00      0.00      0.00       112

    accuracy                           0.87       880
   macro avg       0.44      0.50      0.47       880
weighted avg       0.76      0.87      0.81       880



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
